#1. Data Scraping


In [ ]:
#r/AITA
!gdown 1JPYhjgNh3HfUI51xtAPdlC9dZggr__Fw

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import re
import gensim
import numpy as np
import nltk

# Set the directory path
#directoryA = '/content/drive/MyDrive/Colab Notebooks/dataset' #antonina
directoryD = '/content/drive/MyDrive/Colab Notebooks/AITH_dataset_group_project'
#directoryB = '/content/drive/MyDrive/Colab Notebooks/dataset

# Create the directory if it doesn't exist
if not os.path.exists(directoryD):
    os.makedirs(directoryD)

# Specify the file paths
input_file_path = '/content/drive/MyDrive/Colab Notebooks/AITH_dataset_group_project/AmItheAsshole_submissions_2019_cleaned.json'
output_file_path = '/content/drive/MyDrive/Colab Notebooks/AITH_dataset_group_project/AmItheAsshole_submissions_2019_cleaned_text.json'

# Check if the input file exists
if not os.path.isfile(input_file_path):
    print('Input file does not exist.')
else:
    # Open the file containing the dataset
    with open(input_file_path, 'r') as file:
        data = json.load(file)

In [ ]:
#dont  run this cell, this is for loading data for antonina
# Set the directory path
directoryA = '/content/drive/MyDrive/Colab Notebooks/dataset' #antonina
# Specify the file paths
input_file_path = '/content/drive/MyDrive/Colab Notebooks/dataset/AmItheAsshole_submissions_2019_cleaned.json'

In [ ]:
data[:1]

#2. Data Cleaning



In [ ]:
import pandas as pd
df = pd.read_json(input_file_path )
df.head()

,archived,author,author_created_utc,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,preview,all_awardings,total_awards_received,allow_live_comments,discussion_type,author_premium,awarders,removed_by,removed_by_category,treatment_tags
0,0.0,throwawaysun7,1.546325e+09,None,None,[],None,None,None,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,schbeider44,1.542526e+09,None,None,[],None,None,None,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,-Gashadokuro-,1.521163e+09,None,None,[],None,None,None,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,gchadafish,1.475117e+09,None,None,[],None,None,None,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,swimingduck,1.480811e+09,None,None,[],None,None,None,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

In [ ]:
df["cleaned_text"] = df["selftext"]
df["cleaned_w_punctuation"] = df["selftext"]
df = df[["id","author","selftext", "cleaned_text", 'cleaned_w_punctuation']]

In [ ]:
import re
#prepare the column for topic modelling
# Remove newline characters
df["cleaned_text"] = df["cleaned_text"].str.replace('\n', ' ').str.lower()
# Remove punctuation
df["cleaned_text"] = df["cleaned_text"].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
# Remove numbers
df["cleaned_text"] = df["cleaned_text"].str.replace('\d+', '')
# Remove standalone 'm' and 'f' characters
#df["cleaned_text"] = df["cleaned_text"].str.replace(r'\b[mf]\b', '')

#prepare the column for word embedding
# Remove newline characters and making lower case
df["cleaned_w_punctuation"] = df["cleaned_w_punctuation"].str.replace('\n', ' ').str.lower()
# Remove numbers
df["cleaned_w_punctuation"] = df["cleaned_w_punctuation"].str.replace('\d+', '')
# Remove standalone 'm' and 'f' characters
#df["cleaned_w_punctuation"] = df["cleaned_w_punctuation"].str.replace(r'\b[mf]\b', '')
# remove brackets
df["cleaned_w_punctuation"] = df["cleaned_w_punctuation"].str.replace('(', '')
df["cleaned_w_punctuation"] = df["cleaned_w_punctuation"].str.replace(')', '')

In [ ]:
df['id'] = range(1, len(df) + 1)
df.head(10)

,id,author,selftext,cleaned_text,cleaned_w_punctuation
0,1,throwawaysun7,Throwaway to be safe...\n\nI would be among th...,throwaway to be safe i would be among the ...,throwaway to be safe... i would be among the ...
1,2,schbeider44,My (m24) girlfriend (f26) wants to drink mimos...,my m girlfriend f wants to drink mimosas f...,my m girlfriend f wants to drink mimosas for n...
2,3,-Gashadokuro-,"So, my friends wanted to hang out. The problem...",so my friends wanted to hang out the problem...,"so, my friends wanted to hang out. the problem..."
3,4,gchadafish,So serious dilemma. Tonight for New Years my b...,so serious dilemma tonight for new years my b...,so serious dilemma. tonight for new years my b...
4,5,swimingduck,"In my Bio class, there’s a kid who sits next t...",in my bio class there s a kid who sits next t...,"in my bio class, there’s a kid who sits next t..."
5,6,passionberries,My mom was telling me how I shouldn’t feel bad...,my mom was telling me how i shouldn t feel bad...,my mom was telling me how i shouldn’t feel bad...
6,7,GameGeekFan,My father has not been around for a while and ...,my father has not been around for a while and ...,my father has not been around for a while and ...
7,8,Imadouchebro,*full disclosure my username is just a usernam...,full disclosure my username is just a usernam...,*full disclosure my username is just a usernam...
8,9,stressedinnewark,"My mom can never be happy for anyone, especial...",my mom can never be happy for anyone especial...,"my mom can never be happy for anyone, especial..."
9,10,alburydom,So I’m going away and needed my stick insects ...,so i m going away and needed my stick insects ...,so i’m going away and needed my stick insects ...


In [ ]:
#split data in the column for analysis
#df1 - for topic modelling
# Import pandas library
import pandas as pd

# Define a function that takes a dataframe and a column name as arguments
def split_data(df, col):
  # Create an empty list to store the split data
  split_data = []
  # Loop through each row in the dataframe
  for index, row in df.iterrows():
    # Get the value of the column
    value = row[col]
    # Check if the value contains "however"
    if "however" in value:
      # Split the value at "however"
      parts = value.split("however")
      # Get the part before "however"
      before = parts[0]
      # Get the part after "however"
      after = parts[-1]
      # Append the split data to the list as a tuple
      split_data.append((index, before, after))
  # Create a new dataframe from the list with the specified columns
  df1 = pd.DataFrame(split_data, columns=["id", "before_split", "after_split"])
  # Return the new dataframe
  return df1

# Create a sample dataframe with a column that contains "however"
df_test = pd.DataFrame({'text': ["This is another sentence, however, it has two clauses.", "text however text2 however text 3 however text4"]})

# Print the original dataframe
print("Original dataframe:")
print(df_test)

# Call the function and assign the result to df1
df0 = split_data(df_test, 'text')

# Print the modified dataframe
print("Modified dataframe:")
print(df0)

'''
s="Hey there how are you doing however then I went to the however the other thing that happened."
s.split("however")[0] #change to 0 for the first element
before = [s.split("however")[0]]
s.split("however")[-1] #change to 0 for the last element
after = [s.split("however")[-1]]
'''


Original dataframe:
                                                text
0  This is another sentence, however, it has two ...
1    text however text2 however text 3 however text4
Modified dataframe:
   id                before_split            after_split
0   0  This is another sentence,   , it has two clauses.
1   1                       text                   text4


'\ns="Hey there how are you doing however then I went to the however the other thing that happened."\ns.split("however")[0] #change to 0 for the first element\nbefore = [s.split("however")[0]]\ns.split("however")[-1] #change to 0 for the last element\nafter = [s.split("however")[-1]]\n'

In [ ]:
df0

,id,before_split,after_split
0,0,"This is another sentence,",", it has two clauses."
1,1,text,text4


In [ ]:
#data for topic modelling
# Call the function and assign the result to df1
df1 = split_data(df, 'cleaned_text')
df1.head(10)

,id,before_split,after_split
0,10,while celebrating our nye at a friends party ...,party goers are now annoyed at my husband bec...
1,16,happy new year everyone tl dr at the bottom ...,he came down with some sort of sickness and ...
2,24,sadly this is a serious question in an exclu...,upset because to me the act of bringing con...
3,35,im in th grade right now and have my winter va...,i thought nothing off it wished her happy bi...
4,40,so i ve been dating this girl for about month...,i guess i just thought it would be ok if i c...
5,49,me and my ex boyfriend have been broken up for...,i invited him round and he accepted the offer...
6,53,everyone at this party was new year s at my...,we have a very clear understanding that we d...
7,58,so a couple of really good friends got engaged...,that cost is now starting to effect those of...
8,63,tl dr we had a family cookout for new years ...,my aunts and uncles have never respected my ...
9,64,aita for not giving away my product for free ...,we had some small items that are usually b...


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18154 entries, 0 to 18153
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            18154 non-null  int64 
 1   before_split  18154 non-null  object
 2   after_split   18154 non-null  object
dtypes: int64(1), object(2)
memory usage: 425.6+ KB


# 3. Processing

# 3b. Processing - V2V (micro level: sentence)

Synctactic Parsing

In [ ]:
import nltk

# Download the 'punkt' and 'averaged_perceptron_tagger' resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Function to perform syntactic parsing on a given sentence
def perform_syntactic_parsing(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)

    return tagged

# Perform syntactic parsing on each submission in the data
parsed_data = []
for submission in data:
    parsed_submission = {}
    parsed_submission['title'] = submission.get('title', '')
    parsed_submission['body'] = perform_syntactic_parsing(submission.get('body', ''))
    parsed_data.append(parsed_submission)

# Set the directory path
directory = '/content/drive/MyDrive/Colab Notebooks/dataset'

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Specify the file paths for the original and parsed data
original_file_path = os.path.join(directory, 'AmItheAsshole_submissions_2019_cleaned.json')
parsed_file_path = os.path.join(directory, 'AmItheAsshole_submissions_2019_parsed.json')

# Save the original data to the file
with open(original_file_path, 'w') as file:
    json.dump(data, file)

# Save the parsed data to the file
with open(parsed_file_path, 'w') as file:
    json.dump(parsed_data, file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


NameError: ignored

In [ ]:
# Print the parsed data
print(parsed_data)

[{'title': "AITA for wanting to slowly cut my husband's best friend out of our lives?", 'body': []}, {'title': 'WIBTA not letting girlfriend drink on New Years?', 'body': []}, {'title': 'AITA for telling my friends they shouldn’t be driving on New Year’s Eve at 7:00PM?', 'body': []}, {'title': 'WIBTA for eating Taco Bell...', 'body': []}, {'title': 'AITA for accepting food from this guy?', 'body': []}]


In [ ]:
"Hi there, how are you:".lower().

'hi there, how are you:'

In [ ]:
#from david
from nltk import pos_tag
pos_tag(data[0]["selftext"].split())

In [ ]:
s="Hey there how are you doing however then I went to the however the other thing that happened."
s.split("however")[0] #change to 0 for the first element
before = [s.split("however")[0]]
s.split("however")[-1] #change to 0 for the last element
after = [s.split("however")[-1]]

In [ ]:
before

['Hey there how are you doing ']

In [ ]:
after

[' the other thing that happened.']

In [ ]:
from nltk.toolbox import TreeBuilder
after and before



tldr

#3a. Topic modelling (macro level)